# Prediction Models (intersection)

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

In [29]:
# Import utilities
sys.path.insert(0, '../')
from helpers import preprocessing
from helpers import sets
from helpers import metrics

## Preprocessing

### Import dataset

In [30]:
df = pd.read_csv("../../data_sets/ds2.csv")

### Encode target values

In [31]:
preprocessing.encoding(df)

### Test-Train split

In [32]:
from sklearn.model_selection import train_test_split

subset = sets.pre["inter"][2]

# Input data
X = df[subset]

# Target variable
y = df['y']

# Split dataset in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Scale

In [33]:
pareto = preprocessing.ParetoScaler()
X_train = pareto.fit_transform(X_train)
X_test = pareto.transform(X_test)

## Decision Tree

In [34]:
from sklearn.tree import DecisionTreeClassifier

In [35]:
dt_model = DecisionTreeClassifier(criterion="gini", splitter="best")
dt_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [36]:
y_pred = dt_model.predict(X_test)
metrics.metrics_report(dt_model, X, y, y_test, y_pred)

=== ACCURACY
Mean:		0.7428571428571429
STD:		0.13997084244475302
=== PRECISION
Mean:		0.8166666666666667
STD:		0.1527525231651947
=== RECALL
Mean:		0.82
STD:		0.22271057451320087
=== F1
Mean:		0.7866666666666666
STD:		0.12220201853215576


## Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
lg_model = LogisticRegression(penalty="none")
lg_model.fit(X_train, y_train)

LogisticRegression(penalty='none')

In [12]:
y_pred = lg_model.predict(X_test)
metrics.metrics_report(lg_model, X, y, y_test, y_pred)

=== ACCURACY
Mean:		0.6285714285714284
STD:		0.06998542122237654
=== PRECISION
Mean:		0.6285714285714284
STD:		0.06998542122237654
=== RECALL
Mean:		1.0
STD:		0.0
=== F1
Mean:		0.7696969696969698
STD:		0.051958873331764334


## Naive Bayes

In [13]:
from sklearn.naive_bayes import GaussianNB

In [14]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

GaussianNB()

In [15]:
y_pred = nb_model.predict(X_test)
metrics.metrics_report(nb_model, X, y, y_test, y_pred)

=== ACCURACY
Mean:		0.6571428571428571
STD:		0.1456862718169367
=== PRECISION
Mean:		0.700952380952381
STD:		0.08803215244412456
=== RECALL
Mean:		0.8400000000000001
STD:		0.233238075793812
=== F1
Mean:		0.7432323232323232
STD:		0.13196790908266393


## Support Vector Classifier (SVC)

In [16]:
from sklearn.svm import SVC

In [17]:
svc_model = SVC(kernel="poly", degree=5)
svc_model.fit(X_train, y_train)

SVC(degree=5, kernel='poly')

In [18]:
y_pred = svc_model.predict(X_test)
metrics.metrics_report(svc_model, X, y, y_test, y_pred)

=== ACCURACY
Mean:		0.7142857142857142
STD:		0.20203050891044216
=== PRECISION
Mean:		0.77
STD:		0.139204086785474
=== RECALL
Mean:		0.78
STD:		0.22045407685048604
=== F1
Mean:		0.7651515151515151
STD:		0.17669551196500913


## Artificial Neural Network (ANN)

In [19]:
import tensorflow as tf

2022-08-13 14:42:31.963113: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-13 14:42:31.963141: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [20]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer

In [21]:
def build_model(n_hidden=1, n_neurons=6, learning_rate=3e-3, input_shape=X_train.shape[1]):
  model = Sequential()
  model.add(InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(Dense(n_neurons, activation="relu"))
  model.add(Dense(1, activation='sigmoid'))
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
  return model

In [22]:
from keras.wrappers.scikit_learn import KerasClassifier
ann_model = KerasClassifier(build_model)

/tmp/ipykernel_29182/2775760517.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ann_model = KerasClassifier(build_model)


In [23]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from keras.callbacks import EarlyStopping

param_distribs = {
  "n_hidden": [0, 1, 2, 3],
  "n_neurons": np.arange(1, 10),
  "learning_rate": reciprocal(3e-4, 3e-2),
}
rnd_search_cv = RandomizedSearchCV(ann_model, param_distribs, n_iter=10,
cv=10)
rnd_search_cv.fit( X_train, y_train, batch_size = 32, epochs=100,                  
                  validation_split=0.05,  # using only for earlystopping
                  callbacks=[EarlyStopping(patience=5)])

2022-08-13 14:42:35.762230: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-13 14:42:35.762337: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-13 14:42:35.762406: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tron): /proc/driver/nvidia/version does not exist
2022-08-13 14:42:35.763102: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 23.6422 - accuracy: 0.6087 - val_loss: 9.0817e-17 - val_accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 14.4442 - accuracy: 0.6087 - val_loss: 1.5293e-06 - val_accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 5.2107 - accuracy: 0.6087 - val_loss: 10.4732 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 36ms/step - loss: 5.0312 - accuracy: 0.3913 - val_loss: 18.1373 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 8.7104 - accuracy: 0.3913 - val_loss: 17.1282 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 6.2692 - accuracy: 0.3333
Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 80.5239 - accuracy: 0.6957 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
1/1 [===================

1/1 [==============================] - 1s 1s/step - loss: 420.2764 - accuracy: 0.6522 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 42ms/step - loss: 391.8588 - accuracy: 0.6522 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 44ms/step - loss: 363.6011 - accuracy: 0.6522 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 47ms/step - loss: 335.3675 - accuracy: 0.6522 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 46ms/step - loss: 307.3976 - accuracy: 0.6522 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 539.9964 - accuracy: 0.3333
Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 66.9374 - accuracy: 0.4348 - val_loss: 119.8542 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [======================

Epoch 36/100
1/1 [==============================] - 0s 67ms/step - loss: 3.8585 - accuracy: 0.6087 - val_loss: 3.2724e-05 - val_accuracy: 1.0000
Epoch 37/100
1/1 [==============================] - 0s 44ms/step - loss: 3.9538 - accuracy: 0.6087 - val_loss: 2.9650e-05 - val_accuracy: 1.0000
Epoch 38/100
1/1 [==============================] - 0s 44ms/step - loss: 3.9916 - accuracy: 0.6087 - val_loss: 3.0628e-05 - val_accuracy: 1.0000
Epoch 39/100
1/1 [==============================] - 0s 46ms/step - loss: 3.9792 - accuracy: 0.6087 - val_loss: 3.5482e-05 - val_accuracy: 1.0000
Epoch 40/100
1/1 [==============================] - 0s 43ms/step - loss: 3.9228 - accuracy: 0.6087 - val_loss: 4.5436e-05 - val_accuracy: 1.0000
Epoch 41/100
1/1 [==============================] - 0s 39ms/step - loss: 3.8280 - accuracy: 0.6087 - val_loss: 6.3496e-05 - val_accuracy: 1.0000
Epoch 42/100
1/1 [==============================] - 0s 26ms/step - loss: 3.5779 - accuracy: 0.6667
Epoch 1/100
1/1 [==============

1/1 [==============================] - 0s 31ms/step - loss: 168.1774 - accuracy: 0.0000e+00
Epoch 1/100
1/1 [==============================] - 1s 789ms/step - loss: 539.5483 - accuracy: 0.6522 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 39ms/step - loss: 522.6540 - accuracy: 0.6522 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 505.7771 - accuracy: 0.6522 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 488.9180 - accuracy: 0.6522 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 472.0768 - accuracy: 0.6522 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 25ms/step - loss: 743.0711 - accuracy: 0.3333
Epoch 1/100
1/1 [==============================] - 1s 741ms/step - loss: 523.

...

Epoch 82/100
1/1 [==============================] - 0s 44ms/step - loss: 2555.5955 - accuracy: 0.6250 - val_loss: 2271.5496 - val_accuracy: 0.5000
Epoch 83/100
1/1 [==============================] - 0s 45ms/step - loss: 2551.1350 - accuracy: 0.6250 - val_loss: 2267.5776 - val_accuracy: 0.5000
Epoch 84/100
1/1 [==============================] - 0s 42ms/step - loss: 2546.6750 - accuracy: 0.6250 - val_loss: 2263.6060 - val_accuracy: 0.5000
Epoch 85/100
1/1 [==============================] - 0s 47ms/step - loss: 2542.2148 - accuracy: 0.6250 - val_loss: 2259.6338 - val_accuracy: 0.5000
Epoch 86/100
1/1 [==============================] - 0s 45ms/step - loss: 2537.7546 - accuracy: 0.6250 - val_loss: 2255.6621 - val_accuracy: 0.5000
Epoch 87/100
1/1 [==============================] - 0s 43ms/step - loss: 2533.2947 - accuracy: 0.6250 - val_loss: 2251.6902 - val_accuracy: 0.5000
Epoch 88/100
1/1 [==============================] - 0s 39ms/step - loss: 2528.8345 - accuracy: 0.6250 - val_loss: 2247

Epoch 21/100
1/1 [==============================] - 0s 41ms/step - loss: 0.6670 - accuracy: 0.6522 - val_loss: 0.5908 - val_accuracy: 1.0000
Epoch 22/100
1/1 [==============================] - 0s 37ms/step - loss: 0.6661 - accuracy: 0.6522 - val_loss: 0.5865 - val_accuracy: 1.0000
Epoch 23/100
1/1 [==============================] - 0s 43ms/step - loss: 0.6651 - accuracy: 0.6522 - val_loss: 0.5822 - val_accuracy: 1.0000
Epoch 24/100
1/1 [==============================] - 0s 58ms/step - loss: 0.6642 - accuracy: 0.6522 - val_loss: 0.5781 - val_accuracy: 1.0000
Epoch 25/100
1/1 [==============================] - 0s 51ms/step - loss: 0.6633 - accuracy: 0.6522 - val_loss: 0.5739 - val_accuracy: 1.0000
Epoch 26/100
1/1 [==============================] - 0s 46ms/step - loss: 0.6625 - accuracy: 0.6522 - val_loss: 0.5699 - val_accuracy: 1.0000
Epoch 27/100
1/1 [==============================] - 0s 39ms/step - loss: 0.6617 - accuracy: 0.6522 - val_loss: 0.5659 - val_accuracy: 1.0000
Epoch 28/100


1/1 [==============================] - 0s 45ms/step - loss: 0.6464 - accuracy: 0.6522 - val_loss: 0.4435 - val_accuracy: 1.0000
Epoch 80/100
1/1 [==============================] - 0s 41ms/step - loss: 0.6463 - accuracy: 0.6522 - val_loss: 0.4426 - val_accuracy: 1.0000
Epoch 81/100
1/1 [==============================] - 0s 43ms/step - loss: 0.6463 - accuracy: 0.6522 - val_loss: 0.4417 - val_accuracy: 1.0000
Epoch 82/100
1/1 [==============================] - 0s 43ms/step - loss: 0.6463 - accuracy: 0.6522 - val_loss: 0.4409 - val_accuracy: 1.0000
Epoch 83/100
1/1 [==============================] - 0s 39ms/step - loss: 0.6463 - accuracy: 0.6522 - val_loss: 0.4401 - val_accuracy: 1.0000
Epoch 84/100
1/1 [==============================] - 0s 41ms/step - loss: 0.6462 - accuracy: 0.6522 - val_loss: 0.4393 - val_accuracy: 1.0000
Epoch 85/100
1/1 [==============================] - 0s 41ms/step - loss: 0.6462 - accuracy: 0.6522 - val_loss: 0.4386 - val_accuracy: 1.0000
Epoch 86/100
1/1 [========

1/1 [==============================] - 0s 34ms/step - loss: 30.4024 - accuracy: 0.5652 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 36ms/step - loss: 40.9635 - accuracy: 0.5652 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 37ms/step - loss: 42.1409 - accuracy: 0.5652 - val_loss: 1.5858e-36 - val_accuracy: 1.0000
Epoch 11/100
1/1 [==============================] - 0s 34ms/step - loss: 35.9966 - accuracy: 0.5652 - val_loss: 1.1082e-24 - val_accuracy: 1.0000
Epoch 12/100
1/1 [==============================] - 0s 35ms/step - loss: 24.0154 - accuracy: 0.5652 - val_loss: 4.9657e-08 - val_accuracy: 1.0000
Epoch 13/100
1/1 [==============================] - 0s 22ms/step - loss: 23.3237 - accuracy: 0.0000e+00
Epoch 1/100
1/1 [==============================] - 1s 651ms/step - loss: 138.4299 - accuracy: 0.6250 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
1/1 [================

RandomizedSearchCV(cv=10,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f4f5b1a3cd0>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4f5b1b9730>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([1, 2, 3, 4, 5, 6, 7, 8, 9])})

In [24]:
(rnd_search_cv.best_params_, rnd_search_cv.best_score_)

({'learning_rate': 0.001281202898330887, 'n_hidden': 3, 'n_neurons': 7},
 0.6833333402872086)

In [25]:
ann = rnd_search_cv.best_estimator_.model

In [26]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 10)

Epoch 1/10


2022-08-13 14:47:12.863133: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_BOOL
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_LEGACY_VARIANT
    }
  }
}

	while inferring type of node 'binary_crossentropy/cond/output/_10'


1/1 [==============================] - 1s 783ms/step - loss: 5.4462 - accuracy: 0.6429
Epoch 2/10
1/1 [==============================] - 0s 7ms/step - loss: 5.4462 - accuracy: 0.6429
Epoch 3/10
1/1 [==============================] - 0s 6ms/step - loss: 5.4462 - accuracy: 0.6429
Epoch 4/10
1/1 [==============================] - 0s 7ms/step - loss: 5.4462 - accuracy: 0.6429
Epoch 5/10
1/1 [==============================] - 0s 6ms/step - loss: 5.4462 - accuracy: 0.6429
Epoch 6/10
1/1 [==============================] - 0s 8ms/step - loss: 5.4462 - accuracy: 0.6429
Epoch 7/10
1/1 [==============================] - 0s 10ms/step - loss: 5.4462 - accuracy: 0.6429
Epoch 8/10
1/1 [==============================] - 0s 8ms/step - loss: 5.4462 - accuracy: 0.6429
Epoch 9/10
1/1 [==============================] - 0s 8ms/step - loss: 5.4462 - accuracy: 0.6429
Epoch 10/10
1/1 [==============================] - 0s 7ms/step - loss: 5.4462 - accuracy: 0.6429


In [27]:
metrics.ann_metrics_report(ann, X, y, 5)

1/1 [==============================] - 0s 20ms/step
=== ACCURACY
MEAN (accuracy):		0.6285714285714286
STD (accuracy):		0.45714285714285713
